In [5]:
import pymysql
import json
from functools import partial
import csv
import networkx as nx
import freeman as fm
from random import random, choices
def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro

def multiple_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        res = cursor.fetchall()
        return res

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)

In [11]:
def add_node(g, n):
    g.add_node(n)
    g.nodes[n]['pos'] = (random(), random())
    g.nodes[n]['label'] = n
    g.nodes[n]['color'] = (255, 0, 0)

def add_edge(g, n, m, peso):
    #print("De: ", n, "para: ", m, "peso: ", peso)
    g.add_edge(n, m)
    g[n][m]['weight'] = peso

def get_team_name(connection, id_team):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times WHERE id_time = %s", (id_team))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def get_team_id(connection, nome):
    with connection.cursor() as cursor:
        cursor.execute("SELECT id_time FROM times WHERE nome=%s", (nome))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def list_all_teams(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times")
        res = cursor.fetchall()
        return res
def list_all_transfers(connection, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM transferencia WHERE ano=%s", (ano))
        res = cursor.fetchall()
        return res
def get_from_where(connection, id_jogador, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT times.nome FROM waived INNER JOIN times USING(id_time) WHERE id_jogador = %s AND ano = %s ", (id_jogador, ano))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1

def get_weight(connection, id_time1, id_time2, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time1, id_time2, ano))
        res = cursor.fetchall()
        if res:
            return res
        else: #Pode ser que os nomes estejam invertidos nao sei
            cursor.execute("SELECT peso FROM rivalidade WHERE id_time1=%s AND id_time2=%s AND ano=%s", (id_time2, id_time1, ano))
            if res:
                return res
            return -1


In [12]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2019)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    #else:
        #print(antigo, novo)
    
    
a.rec(g)
a.play()

In [18]:


a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2018)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    else:
        print(antigo, novo)
    
    
a.rec(g)
a.play()


-1 ('LA Clippers',)
-1 ('New Orleans Pelicans',)
-1 ('New Orleans Pelicans',)
-1 ('Washington Wizards',)
-1 ('Atlanta Hawks',)
-1 ('Los Angeles Lakers',)
-1 ('Philadelphia 76ers',)
-1 ('Detroit Pistons',)
-1 ('Toronto Raptors',)
-1 ('Orlando Magic',)
-1 ('Indiana Pacers',)
-1 ('Brooklyn Nets',)
-1 ('Minnesota Timberwolves',)
-1 ('Indiana Pacers',)
-1 ('Memphis Grizzlies',)
-1 ('Memphis Grizzlies',)
-1 ('New Orleans Pelicans',)
-1 ('Houston Rockets',)
-1 ('Houston Rockets',)
-1 ('Dallas Mavericks',)
-1 ('Los Angeles Lakers',)
-1 ('Atlanta Hawks',)
-1 ('Atlanta Hawks',)
-1 ('Golden State Warriors',)
-1 ('Atlanta Hawks',)
-1 ('Atlanta Hawks',)
-1 ('Oklahoma City Thunder',)
-1 ('Dallas Mavericks',)
-1 ('Dallas Mavericks',)
-1 ('Indiana Pacers',)
-1 ('New Orleans Pelicans',)
-1 ('New Orleans Pelicans',)
-1 ('Atlanta Hawks',)
-1 ('Los Angeles Lakers',)
-1 ('LA Clippers',)
-1 ('San Antonio Spurs',)
-1 ('Brooklyn Nets',)
-1 ('New Orleans Pelicans',)
-1 ('San Antonio Spurs',)
-1 ('Minnesota Tim

In [17]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2017)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    else:
        print(antigo, novo)
    
    
a.rec(g)
a.play()

-1 ('San Antonio Spurs',)
-1 ('Sacramento Kings',)
-1 ('Orlando Magic',)
-1 ('Denver Nuggets',)
-1 ('Orlando Magic',)
-1 ('LA Clippers',)
-1 ('Brooklyn Nets',)
-1 ('Toronto Raptors',)
-1 ('San Antonio Spurs',)
-1 ('Philadelphia 76ers',)
-1 ('Phoenix Suns',)
-1 ('Sacramento Kings',)
-1 ('Toronto Raptors',)
-1 ('Boston Celtics',)
-1 ('Milwaukee Bucks',)
-1 ('Phoenix Suns',)
-1 ('Milwaukee Bucks',)
-1 ('Dallas Mavericks',)
-1 ('New Orleans Pelicans',)
-1 ('New Orleans Pelicans',)
-1 ('Sacramento Kings',)
-1 ('Golden State Warriors',)
-1 ('Indiana Pacers',)
-1 ('Portland Trail Blazers',)
-1 ('Dallas Mavericks',)
-1 ('San Antonio Spurs',)
-1 ('Golden State Warriors',)
-1 ('Houston Rockets',)
-1 ('Utah Jazz',)
-1 ('Charlotte Hornets',)
-1 ('New Orleans Pelicans',)
-1 ('Charlotte Hornets',)
-1 ('Houston Rockets',)
-1 ('Philadelphia 76ers',)
-1 ('Charlotte Hornets',)
-1 ('Washington Wizards',)
-1 ('Boston Celtics',)
-1 ('Washington Wizards',)
-1 ('Atlanta Hawks',)
-1 ('Sacramento Kings',)
-1 (

In [15]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2016)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    else:
        print(antigo, novo)
    
    
a.rec(g)
a.play()

In [16]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection, 2015)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    antigo = get_from_where(connection, transfer[1], transfer[3])
    novo = get_team_name(connection, transfer[2])
    if(antigo != -1) and (novo != -1):
        if(antigo != novo):
            id_antigo = get_team_id(connection, antigo)
            id_novo = get_team_id(connection, novo)
            weight = get_weight(connection, id_antigo, id_novo, 2019)
            if(weight == -1):
                weight = 0
            add_edge(g, antigo, novo, weight)
    #else:
        #print(antigo, novo)
    
    
a.rec(g)
a.play()

In [4]:
connection.close()